In [9]:
# Cell 1: Import necessary libraries
import numpy as np
import pandas as pd
import torch


# Add your project path if needed
# sys.path.append('/path/to/your/project')

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

PyTorch version: 2.6.0
CUDA available: False


In [10]:
# Cell 2: Import the NBEATSx implementation

import sys
import os

try:
    # Adjust these imports based on your actual file structure
    from nbeats import Nbeats
    from nbeats_model import NBeats, NBeatsBlock, TrendBasis, SeasonalityBasis, IdentityBasis
    print("✅ Successfully imported NBEATSx components")
except ImportError as e:
    print("❌ Import error:", e)
    print("Please check your file paths and imports")

✅ Successfully imported NBEATSx components


In [11]:
# Cell 3: Create synthetic time series data (mimicking your volatility targets)
def create_synthetic_volatility_data(n_samples=1000, input_size=60, output_size=12):
    """
    Create synthetic data that resembles your volatility targets
    """
    np.random.seed(42)
    
    # Create time series with trend + seasonality + noise (like VIX term structure)
    time = np.arange(n_samples + input_size + output_size)
    
    # Base trend
    trend = 0.01 * time + 10
    
    # Seasonal component (quarterly patterns in volatility)
    seasonal = 3 * np.sin(2 * np.pi * time / 63)  # ~3 month cycle
    
    # Add volatility clustering (like real financial data)
    noise = np.random.randn(len(time)) * 0.5
    garch_vol = np.zeros(len(time))
    garch_vol[0] = 1
    for i in range(1, len(time)):
        garch_vol[i] = 0.1 + 0.05 * noise[i-1]**2 + 0.9 * garch_vol[i-1]
    
    # Combine components
    series = trend + seasonal + noise * garch_vol
    
    # Create input-output pairs
    X = []
    y = []
    
    for i in range(n_samples):
        X.append(series[i:i+input_size])
        y.append(series[i+input_size:i+input_size+output_size])
    
    return np.array(X), np.array(y)

# Generate test data
X_test, y_test = create_synthetic_volatility_data(n_samples=500, input_size=60, output_size=12)

print(f"Generated synthetic data:")
print(f"X shape: {X_test.shape}")  # Should be (500, 60)
print(f"y shape: {y_test.shape}")  # Should be (500, 12)
print(f"X sample stats: mean={X_test.mean():.2f}, std={X_test.std():.2f}")

Generated synthetic data:
X shape: (500, 60)
y shape: (500, 12)
X sample stats: mean=12.79, std=2.60


In [12]:
# Cell 4: Create a simple data loader class
class SimpleTimeSeriesLoader:
    """
    Simplified data loader that mimics the original TimeSeriesLoader interface
    """
    def __init__(self, X, y, batch_size=32, shuffle=True):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.input_size = X.shape[1]
        self.output_size = y.shape[1]
        self.n_samples = len(X)
        
        # Create indices
        self.indices = np.arange(self.n_samples)
        if shuffle:
            np.random.shuffle(self.indices)
        
        self.current_idx = 0
    
    def __iter__(self):
        self.current_idx = 0
        if self.shuffle:
            np.random.shuffle(self.indices)
        return self
    
    def __next__(self):
        if self.current_idx >= self.n_samples:
            raise StopIteration
        
        # Get batch indices
        end_idx = min(self.current_idx + self.batch_size, self.n_samples)
        batch_indices = self.indices[self.current_idx:end_idx]
        
        # Create batch
        batch_X = self.X[batch_indices]
        batch_y = self.y[batch_indices]
        
        # Create batch dictionary matching original interface
        batch = {
            'insample_y': batch_X,  # Historical data
            'insample_x': np.zeros((len(batch_indices), 1, self.input_size)),  # Dummy exogenous
            'insample_mask': np.ones((len(batch_indices), self.input_size)),  # No missing data
            'outsample_x': np.zeros((len(batch_indices), 1, self.output_size)),  # Dummy exogenous
            'outsample_y': batch_y,  # Target data
            'outsample_mask': np.ones((len(batch_indices), self.output_size)),  # No missing data
            's_matrix': np.zeros((len(batch_indices), 0))  # No static features
        }
        
        self.current_idx = end_idx
        return batch
    
    def get_n_variables(self):
        # Return (n_temporal_exogenous, n_static_exogenous)
        return 1, 0  # Simplified - just the target variable

# Create data loaders
train_size = int(0.8 * len(X_test))
X_train, X_val = X_test[:train_size], X_test[train_size:]
y_train, y_val = y_test[:train_size], y_test[train_size:]

train_loader = SimpleTimeSeriesLoader(X_train, y_train, batch_size=32, shuffle=True)
val_loader = SimpleTimeSeriesLoader(X_val, y_val, batch_size=32, shuffle=False)

print(f"Created data loaders:")
print(f"Train samples: {len(X_train)}")
print(f"Val samples: {len(X_val)}")
print(f"Input size: {train_loader.input_size}")
print(f"Output size: {train_loader.output_size}")

Created data loaders:
Train samples: 400
Val samples: 100
Input size: 60
Output size: 12


In [15]:
# Cell 5: Test NBEATSx model creation
try:
    # Create NBEATSx model with simplified configuration
    # Fix: The original code expects single values for harmonics/polynomials per stack
    model_config = {
        'input_size_multiplier': 5,  # This means input_size = 5 * output_size = 60
        'output_size': 12,  # Forecast horizon
        'shared_weights': False,
        'activation': 'relu',
        'initialization': 'he_uniform',
        'stack_types': ['trend', 'seasonality', 'identity'],
        'n_blocks': [2, 2, 1],  # Number of blocks per stack
        'n_layers': [4, 4, 4],  # Number of layers per stack
        'n_hidden': [[256, 256, 256, 256], [256, 256, 256, 256], [256, 256, 256, 256]],  # Hidden units per layer
        # Fix: These should be single values, not lists
        'n_harmonics': 5,  # For seasonality stack (single value)
        'n_polynomials': 3,  # For trend stack (single value)
        'exogenous_n_channels': 0,  # No complex exogenous variables
        'include_var_dict': None,  # Simplified
        't_cols': None,  # Simplified
        'batch_normalization': False,
        'dropout_prob_theta': 0.1,
        'dropout_prob_exogenous': 0.1,
        'x_s_n_hidden': 0,  # No static features
        'learning_rate': 0.001,
        'lr_decay': 0.5,
        'n_lr_decay_steps': 3,
        'weight_decay': 1e-4,
        'l1_theta': 0.0,
        'n_iterations': 100,  # Just for quick test
        'early_stopping': 10,
        'loss': 'MAE',
        'loss_hypar': None,
        'val_loss': 'MAE',
        'random_seed': 42,
        'seasonality': 7,  # Daily seasonality
        'device': 'cuda' if torch.cuda.is_available() else 'cpu'
    }
    
    # Create model
    nbeats_model = Nbeats(**model_config)
    
    print("✅ Successfully created NBEATSx model")
    print(f"Model device: {nbeats_model.device}")
    print(f"Input size: {nbeats_model.input_size}")
    print(f"Output size: {nbeats_model.output_size}")
    print(f"Stack types: {nbeats_model.stack_types}")
    print(f"N harmonics: {nbeats_model.n_harmonics}")
    print(f"N polynomials: {nbeats_model.n_polynomials}")
    
except Exception as e:
    print("❌ Error creating model:", e)
    import traceback
    traceback.print_exc()

✅ Successfully created NBEATSx model
Model device: cpu
Input size: 60
Output size: 12
Stack types: ['trend', 'seasonality', 'identity']
N harmonics: 5
N polynomials: 3


In [17]:
# Cell 6: Test model training (quick test)
try:
    print("Starting model training test...")
    
    # Train for just a few iterations to test
    nbeats_model.fit(
        train_ts_loader=train_loader,
        val_ts_loader=val_loader,
        n_iterations=50,  # Very short for quick test
        verbose=True,
        eval_steps=10
    )
    
    print("✅ Training test completed successfully!")
    
except Exception as e:
    print("❌ Error during training:", e)
    import traceback
    traceback.print_exc()

Starting model training test...
❌ Error during training: module 'numpy' has no attribute 'float'.
`np.float` was a deprecated alias for the builtin `float`. To avoid this error in existing code, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Traceback (most recent call last):
  File "/var/folders/31/b1v52j156gbfz83pmrnth3jc0000gn/T/ipykernel_121/2736211935.py", line 6, in <module>
    nbeats_model.fit(
    ~~~~~~~~~~~~~~~~^
        train_ts_loader=train_loader,
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<3 lines>...
        eval_steps=10
        ^^^^^^^^^^^^^
    )
    ^
  File "/Users/jamesliu/Documents/GitHub/bloomberg-metalearning/models/base/nbeatsx/nbeats.py", line 410, in fit
    block_list = self.create_stack()
  File "/Users/jamesliu/Documents/GitHub/bloomberg-metalearning/models/base/nbeatsx/nbeats.py", line 262, in create_stack
    basis=TrendBasis(degree_of_polynomial=self.n_polynomials,
          ~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
                             backcast_size=self.input_size,
                             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
                             forecast_size=self.output_size),
                             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jamesliu